<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/hands_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe==0.10.5

In [ ]:
import os
# hand module
if not os.path.exists("hand_landmarker.task"):
  !wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task
# pose module
if not os.path.exists("pose_landmarker.task"):
  !wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

In [ ]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os

In [3]:
# load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
test_folder = folder + "test/"
training_video = training_folder + "video/"
test_video = test_folder + "video/"
training_images = training_folder + "images/"
test_images = test_folder + "images/"

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']

In [ ]:
import ast
splitted = pd.read_csv("/content/drive/MyDrive/NLP/splitted.csv")
splitted['train'] = splitted['train'].apply(ast.literal_eval)
splitted['test'] = splitted['test'].apply(ast.literal_eval)

Holistic model can detect pose, hands and face features.
-  pose: 33 keypoints, we will save the landmarks (x, y, z, visibility)
- left hand: 21 keypoints, we will save the landmarks (x, y, z)
- right hand: 21 keypoints, we will save the landmarks (x, y, z)
- face (not used): 468 keypoints


In [ ]:
# holistic model

mp_holistic = mp.solutions.holistic

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):
    video_id = js_file['instances'][i][j]['video_id']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    frame_end = js_file['instances'][i][j]['frame_end']
    frame_start = js_file['instances'][i][j]['frame_start']
    filename = f"{word}_{video_id}.mp4"

    if source not in youtube_videos:
      source_path = training_video if split == 'train' else test_video
      dest_path = training_images if split == 'train' else test_images
      os.makedirs(dest_path, exist_ok=True)
      dest_file = f"landmarks_{word}_{video_id}.npy"
      if dest_file not in dest_path:
        cap = cv2.VideoCapture(os.path.join(source_path, filename))

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_end = frame_end if frame_end !=-1 else total_frames
        all_frame_features = []
        curr_frame = 0

        with mp_holistic.Holistic(
          min_detection_confidence=0.5,
          min_tracking_confidence=0.5) as holistic:

          while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
              break

            if curr_frame < frame_start:
              curr_frame += 1
              continue

            if frame_end != -1 and curr_frame > frame_end:
              break
            frame_features = []

            # to improve performance, optionally mark the image as not writable to pass by inference
            frame.flags.writeable = False
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)

            # save landmarks
            if results.pose_landmarks:
              for landmark in results.pose_landmarks.landmark:
                frame_features.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            else:
              frame_features.extend([0.0]*132) # 33 points, each with 4 values (x, y, z, visibility)

            if results.left_hand_landmarks:
              for landmark in results.left_hand_landmarks.landmark:
                frame_features.extend([landmark.x, landmark.y, landmark.z])
            else:
              frame_features.extend([0.0]*63) # points, each with 3 values (x, y, z)

            if results.right_hand_landmarks:
              for landmark in results.right_hand_landmarks.landmark:
                frame_features.extend([landmark.x, landmark.y, landmark.z])
            else:
              frame_features.extend([0.0]*63) # points, each with 3 values (x, y, z)

            all_frame_features.append(frame_features) # for the whole video

            curr_frame += 1

          cap.release()

          np.save(os.path.join(dest_path, dest_file), np.array(all_frame_features))

glosses:   5%|▌         | 109/2000 [1:31:55<26:26:23, 50.33s/it]

In [ ]:
# holistic model sample
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

sample = training_video + "adjective_01066.mp4"
i, j = 1603, 3
cap = cv2.VideoCapture(sample)
frame_end = js_file['instances'][int(i)][int(j)]['frame_end']
frame_start = js_file['instances'][int(i)][int(j)]['frame_start']
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_end = frame_end if frame_end !=-1 else total_frames
step_frame = (frame_end - frame_start + 1) // 16
curr_frame = 0
printed_frames = 0
all_frame_features = []

with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
        continue
      if curr_frame < frame_start:
        curr_frame += 1
      if frame_end != -1 and curr_frame > frame_end:
        break
      frame_features = []
      # to improve performance, optionally mark the image as not writable to pass by inference
      frame.flags.writeable = False
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = holistic.process(image)

      # save landmarks
      if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
          frame_features.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
      else:
        frame_features.extend([0.0]*132) # 33 points, each with 4 values (x, y, z, visibility)

      if results.left_hand_landmarks:
        for landmark in results.left_hand_landmarks.landmark:
          frame_features.extend([landmark.x, landmark.y, landmark.z])
      else:
        frame_features.extend([0.0]*63) # points, each with 3 values (x, y, z)

      if results.right_hand_landmarks:
        for landmark in results.right_hand_landmarks.landmark:
          frame_features.extend([landmark.x, landmark.y, landmark.z])
      else:
        frame_features.extend([0.0]*63) # points, each with 3 values (x, y, z)

      all_frame_features.append(frame_features) # for the whole video
#np.save(f"/content/drive/MyDrive/NLP/dataset/SPLIT/images/landmarks_{video_id}.npy", all_frame_features)


In [ ]:
# visualization pose estimations

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [ ]:
# pose detection

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):
    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    frame_end = js_file['instances'][i][j]['frame_end']
    frame_start = js_file['instances'][i][j]['frame_start']
    filename = f"{word}_{video_id}.mp4"

    if source not in youtube_videos:
      image_path = training_images if split == 'train' else test_images
      pose_folder = image_path + "pose/"
      os.makedirs(pose_folder, exist_ok=True)
      path = training_video if split == 'train' else test_video
      sample = path + filename
      cap = cv2.VideoCapture(sample)
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      frame_end = frame_end if frame_end !=-1 else total_frames
      step_frame = (frame_end - frame_start + 1) // 8
      curr_frame = 0
      printed_frames = 0

      # STEP 2: Create an PoseLandmarker object.
      base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
      options = vision.PoseLandmarkerOptions(
          base_options=base_options,
          output_segmentation_masks=True)
      detector = vision.PoseLandmarker.create_from_options(options)

      while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
          break
        if curr_frame < frame_start:
          curr_frame += 1
        if frame_end != -1 and curr_frame > frame_end:
          break
        if (curr_frame - frame_start) % step_frame == 0 and printed_frames < 8:
          curr_filename = f"{word}_{video_id}_{printed_frames}.jpg"
          if curr_filename not in os.listdir(pose_folder):
            output_path = pose_folder + curr_filename
            data = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # STEP 3: Load the input image.
            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=data)
            # STEP 4: Detect hand landmarks from the input image.
            detection_result = detector.detect(image)
            # STEP 5: Process the classification result. In this case, visualize it.
            annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
            cv2.imwrite(output_path, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
            printed_frames += 1
        curr_frame += 1

# details about the landmarks https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker

In [ ]:
# visualization hand estimation

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
# hand detection

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):
    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    frame_end = js_file['instances'][i][j]['frame_end']
    frame_start = js_file['instances'][i][j]['frame_start']
    filename = f"{word}_{video_id}.mp4"

    if source not in youtube_videos:
      image_path = training_images if split == 'train' else test_images
      hand_folder = image_path + "hand/"
      os.makedirs(hand_folder, exist_ok=True)
      path = training_video if split == 'train' else test_video
      sample = path + filename
      cap = cv2.VideoCapture(sample)
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      frame_end = frame_end if frame_end !=-1 else total_frames
      step_frame = (frame_end - frame_start + 1) // 16
      curr_frame = 0
      printed_frames = 0

      # STEP 2: Create an HandLandmarker object.
      base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
      options = vision.HandLandmarkerOptions(base_options=base_options,
                                            num_hands=2)
      detector = vision.HandLandmarker.create_from_options(options)

      while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
          break
        if curr_frame < frame_start:
          curr_frame += 1
        if frame_end != -1 and curr_frame > frame_end:
          break
        if (curr_frame - frame_start) % step_frame == 0 and printed_frames < 16:
          curr_filename = f"{word}_{video_id}_{printed_frames}.jpg"
          if curr_filename not in os.listdir(hand_folder):
            output_path = hand_folder + curr_filename
            data = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # STEP 3: Load the input image.
            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=data)
            # STEP 4: Detect hand landmarks from the input image.
            detection_result = detector.detect(image)
            # STEP 5: Process the classification result. In this case, visualize it.
            annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
            cv2.imwrite(output_path, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
            printed_frames += 1
        curr_frame += 1

# details about the landmarks https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker

In [ ]:
# hand detection sample

sample = training_video + "adjective_01066.mp4"
i, j = 1603, 3
cap = cv2.VideoCapture(sample)
frame_end = js_file['instances'][int(i)][int(j)]['frame_end']
frame_start = js_file['instances'][int(i)][int(j)]['frame_start']
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_end = frame_end if frame_end !=-1 else total_frames
step_frame = (frame_end - frame_start + 1) // 16
curr_frame = 0
printed_frames = 0

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break
  if curr_frame < frame_start:
    curr_frame += 1
  if frame_end != -1 and curr_frame > frame_end:
    break
  if (curr_frame - frame_start) % step_frame == 0 and printed_frames < 32:
    data = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # STEP 3: Load the input image.
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=data)
    # STEP 4: Detect hand landmarks from the input image.
    detection_result = detector.detect(image)
    # STEP 5: Process the classification result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    landmarks = detection_result.hand_landmarks
    if landmarks != []:
      # print only images in which the hand(s) has(ve) been detected
      cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
      for idx, hand_landmarks in enumerate(landmarks):
        x = [landmark.x for landmark in hand_landmarks] # Normalized x coordinate [0.0, 1.0]
        y = [landmark.y for landmark in hand_landmarks] # Normalized x coordinate [0.0, 1.0]
        z = [landmark.z for landmark in hand_landmarks] # Relative depth
        visibility = [landmark.visibility for landmark in hand_landmarks] # Confidence score of the landmark visibility

    printed_frames += 1
  curr_frame += 1